# Cài Co-DETR

In [ ]:
!git clone https://github.com/Sense-X/Co-DETR.git
!pip install -e /kaggle/working/Co-DETR
!mv /kaggle/working/Co-DETR /kaggle/working/Co_DETR

In [ ]:
!nvcc --version
import torch
print(torch.__version__)

In [ ]:
# 1. Tải và cài CUDA 11.6.2
!wget https://developer.download.nvidia.com/compute/cuda/11.6.2/local_installers/cuda_11.6.2_510.47.03_linux.run
!chmod +x cuda_11.6.2_510.47.03_linux.run
!sudo sh cuda_11.6.2_510.47.03_linux.run --silent --toolkit

# 2. Thiết lập biến môi trường
import os
os.environ['PATH'] += ':/usr/local/cuda-11.6/bin'
os.environ['CUDA_HOME'] = '/usr/local/cuda-11.6'

# 3. Cài đặt PyTorch 1.12.1 với CUDA 11.6
!pip install torch==1.12.1+cu116 torchvision==0.13.1+cu116 torchaudio==0.12.1 --extra-index-url https://download.pytorch.org/whl/cu116

# 4. Cài đặt mmcv-full==1.6.2 (build cho torch 1.12.1 + cu116)
!pip install mmcv-full==1.6.2 -f https://download.openmmlab.com/mmcv/dist/cu116/torch1.12.1/index.html


In [ ]:
# !pip install torch==1.12.1+cu116 torchvision==0.13.1+cu116 -f https://download.pytorch.org/whl/cu116/torch_stable.html

In [ ]:
!nvcc --version
import torch
print(torch.__version__)

In [ ]:
!pip uninstall -y mmcv-full mmcv openmim
!pip uninstall -y mmcv mmcv-full mim openmim
!rm -rf /opt/conda/lib/python3.10/site-packages/mmcv*


In [ ]:
pip install mmcv-full==1.6.2 -f https://download.openmmlab.com/mmcv/dist/cu116/torch1.12/index.html

In [ ]:
import mmcv
print(mmcv.__version__)


In [ ]:
import torch
from mmcv import Config
import sys
sys.path.append('/kaggle/working/Co_DETR')
from Co_DETR.mmdet.apis import init_detector, inference_detector
import cv2
import numpy as np
from sklearn.cluster import KMeans
from sklearn.neighbors import KDTree

def infer_one_image(config_path, ckpt_path, image_path, score_thr=0.5, device='cuda'):
    cfg = Config.fromfile(config_path)
    model = init_detector(cfg, ckpt_path, device=device)

    result = inference_detector(model, image_path)
    img = cv2.imread(image_path)

    basic_colors = {
        'red': (255, 0, 0), 'green': (0, 255, 0), 'blue': (0, 0, 255),
        'yellow': (255, 255, 0), 'cyan': (0, 255, 255), 'magenta': (255, 0, 255),
        'black': (0, 0, 0), 'white': (255, 255, 255), 'gray': (128, 128, 128),
        'orange': (255, 165, 0), 'brown': (165, 42, 42), 'pink': (255, 192, 203),
        'purple': (128, 0, 128)
    }
    color_names = list(basic_colors.keys())
    color_values = np.array(list(basic_colors.values()))
    tree = KDTree(color_values)

    if isinstance(result, tuple):
        result = result[0]  # bỏ segmentation mask nếu có

    for class_id, class_result in enumerate(result):
        for bbox in class_result:
            x1, y1, x2, y2, score = bbox
            class_name = model.CLASSES[class_id]
            if score < score_thr:
                continue
            # Crop object từ ảnh gốc
            x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])
            crop = img[y1:y2, x1:x2]
            if crop.size == 0:
                continue

            crop_rgb = cv2.cvtColor(crop, cv2.COLOR_BGR2RGB).reshape(-1, 3)
            kmeans = KMeans(n_clusters=1, random_state=42).fit(crop_rgb)
            dom_color = kmeans.cluster_centers_[0].astype(int)
            dist, idx = tree.query([dom_color], k=1)
            color_name = color_names[idx[0][0]]

            bbox_color = tuple(int(c) for c in dom_color[::-1])
            cv2.rectangle(img, (x1, y1), (x2, y2), bbox_color, thickness=3)

            cv2.putText(img, f"{color_name} {class_name}", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 0), thickness=4, lineType=cv2.LINE_AA)
            cv2.putText(img, f"{color_name} {class_name}", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, bbox_color, thickness=2, lineType=cv2.LINE_AA)

            print(f"{class_name} | score={score:.2f} | dominant color={dom_color.tolist()} | nearest color={color_name} | bbox=({x1},{y1},{x2},{y2})")

    out_file = 'result_with_colors.jpg'
    cv2.imwrite(out_file, img)
    print(f"Saved result with colored bboxes to {out_file}")

if __name__ == '__main__':
    config_path = r"/kaggle/working/Co_DETR/projects/configs/co_dino/co_dino_5scale_swin_large_16e_o365tococo.py"
    ckpt_path = r"/kaggle/input/alockpt/co_dino_5scale_swin_large_16e_o365tococo.pth"
    image_path = r"/kaggle/input/anhchomeo/anh-cho-meo.jpg"
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    infer_one_image(config_path, ckpt_path, image_path, score_thr=0.5, device=device)


In [ ]:
!nvidia-smi
import torch
print(torch.__version__)
print(torch.cuda.is_available())
print(torch.version.cuda)
